In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/Hackathon-BBVA-2022
!ls -a

/content/drive/MyDrive/Github/Hackathon-BBVA-2022
CHANGELOG.rst  House_prediction.ipynb  README.md
.git	       LICENSE		       requirements.txt


In [3]:
!git checkout enzo

Already on 'enzo'


In [11]:
!git status

On branch enzo
Your branch is up to date with 'origin/enzo'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   House_prediction.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   House_prediction.ipynb



In [12]:
!git add House_prediction.ipynb

In [ ]:
!git config --global user.email "enzo.cisneros@pucp.edu."

In [13]:
!git commit -m "0.0.2"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@4370ef63c95f.(none)')


In [14]:
!git status

On branch enzo
Your branch is up to date with 'origin/enzo'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   House_prediction.ipynb



In [15]:
!git push --set-upstream origin enzo

Branch 'enzo' set up to track remote branch 'enzo' from 'origin'.
Everything up-to-date
